# Objetivo do projeto
Compreender as áreas de menor cobertura da empresa com o intuito de direcionar os esforços de expansão de cobertura.


In [ ]:
!pip install plotly_express
!pip install pycountry-convert
!pip install geopandas
!pip install geopy

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly_express as px
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
import numpy as np
from geopy.geocoders import Nominatim

In [ ]:
# Lendo o dataset 
df = pd.read_excel('/content/drive/MyDrive/deals-14902126-85.xlsx')

In [ ]:
# Verificando as primeiras 5 linhas
df.head()

,Deal - Title,Deal - Organisation,Deal - Deal created,Deal - Value,Deal - Owner,Deal - Industry,Deal - Methodology,Deal - Location,Deal - Sample,Deal - Research Type,Deal - Category,Deal - LOI (in minutes),Deal - Lost reason,Deal - Lost time,Deal - Stage
0,IRB - IDIs with HCPs in US,IRB,2022-05-04 09:56:02,0,Amon Martins,Healthcare,IDI,United States of America,15.0,Qualitative,NaN,NaN,Falta de cobertura,2022-05-05 15:23:06,Project Received
1,PMR0811 | Smart,Philomath Research,2022-01-26 00:00:00,0,Rafaela Freitas,B2B,CAWI,United States of America,600.0,Quantitative,NaN,NaN,Falta de cobertura,2022-05-05 11:33:46,Proposal Made
2,Sample for Postal Measurement Panel - Netherlands,Kantar,2022-04-04 00:00:00,0,Rafaela Freitas,B2C,CAWI,Netherlands,318.0,Quantitative,NaN,NaN,Falta de cobertura,2022-05-05 11:36:33,Project Received
3,AIRBUS EMPLOYEES RFQ,IRB,2022-04-18 00:00:00,0,Rafaela Freitas,B2B,IDI,"France, Italy, United Kingdom, United States o...",105.0,Qualitative,NaN,NaN,Falta de cobertura,2022-05-05 11:37:19,Project Received
4,Multi-country study in Middle East qual,IRB,2022-04-27 00:00:00,0,Walter Tapias,B2C,IDI,"Saudi Arabia, United Arab Emirates, Qatar, Kuwait",0.0,Qualitative,NaN,NaN,Falta de cobertura,2022-05-05 13:28:58,Project Received


In [ ]:
# Verificando as últimas 5 linhas
df.tail()

,Deal - Title,Deal - Organisation,Deal - Deal created,Deal - Value,Deal - Owner,Deal - Industry,Deal - Methodology,Deal - Location,Deal - Sample,Deal - Research Type,Deal - Category,Deal - LOI (in minutes),Deal - Lost reason,Deal - Lost time,Deal - Stage
110,Sermo RfQ - Acromegaly ATU - Colombia,Sermo,2022-12-15 08:47:13,0,Kevin Bezerra,Healthcare,CAWI,Colombia,40.0,Quantitative,NaN,30.0,Falta de cobertura,2022-12-16 09:11:48,Project Received
111,Food Store Managers,Focus Insite,2022-12-16 11:53:47,0,Amon Martins,B2B,CATI,"Canada, United States of America",100.0,Quantitative,NaN,10.0,Falta de cobertura,2022-12-16 11:54:06,Project Received
112,BA151: Pricing Request: Skin Care Request: Sma...,Borderless Access,2022-12-21 08:59:32,0,Lamoni Queiroz,B2C,FGD,Mexico,6.0,Qualitative,NaN,120.0,Falta de cobertura,2022-12-23 09:59:48,Qualified
113,P22-66060,Market xcel,2022-12-22 08:12:10,51000,Lamoni Queiroz,B2C,"Diary Study, CAWI",United States of America,300.0,Quantitative,NaN,15.0,Falta de cobertura,2022-12-23 10:01:16,Proposal Made
114,22-2026 Market Consumption US,Focus Insite,2022-12-22 15:20:01,0,Amon Martins,B2C,CAWI,United States of America,500.0,Quantitative,NaN,NaN,Falta de cobertura,2022-12-22 15:20:24,Project Received


In [ ]:
# Extraindo informações básicas do dataframe

df['Deal - Industry'] = df['Deal - Industry'].fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Deal - Title             115 non-null    object 
 1   Deal - Organisation      115 non-null    object 
 2   Deal - Deal created      115 non-null    object 
 3   Deal - Value             115 non-null    int64  
 4   Deal - Owner             115 non-null    object 
 5   Deal - Industry          115 non-null    object 
 6   Deal - Methodology       97 non-null     object 
 7   Deal - Location          93 non-null     object 
 8   Deal - Sample            90 non-null     float64
 9   Deal - Research Type     95 non-null     object 
 10  Deal - Category          7 non-null      object 
 11  Deal - LOI (in minutes)  54 non-null     float64
 12  Deal - Lost reason       115 non-null    object 
 13  Deal - Lost time         115 non-null    object 
 14  Deal - Stage             1

In [ ]:
# Separando os valores de "Deal - Location" para contar quantos projetos foram perdidos em cada país individualmente.

new_df = df.set_index(['Deal - Title', 'Deal - Organisation', 'Deal - Methodology', 'Deal - Sample', 'Deal - Industry', 'Deal - Value', 'Deal - Research Type'])['Deal - Location'].str.split(', ', expand=True).stack().reset_index().rename(columns={0: 'Deal - Location'})

# Agrupando e explorando quantidade de projetos perdidos por localidade

In [ ]:
# Agrupando e contando o número de projetos perdidos em cada país.

grouped = new_df.groupby('Deal - Location')['Deal - Title'].count().reset_index()

grouped

,Deal - Location,Deal - Title
0,Argentina,1
1,Australia,7
2,Azerbaijan,2
3,Bahamas,2
4,Belgium,1
5,Belice,1
6,Brazil,5
7,Canada,6
8,Chile,2
9,China,6


In [ ]:
# Colocando em ordem de mais projetos perdidos para menos

gro_sor = grouped.sort_values('Deal - Title', ascending=False)
gro_sor

,Deal - Location,Deal - Title
51,United States of America,45
15,France,12
17,Germany,9
50,United Kingdom,9
1,Australia,7
24,Italy,6
9,China,6
45,Spain,6
7,Canada,6
21,India,6


# Indústrias


## B2C

In [ ]:
# Filtrando projetos perdidos de acordo com a indústria

df_b2c = new_df[new_df['Deal - Industry'].str.contains("B2C", na=False)]
df_b2c

,Deal - Title,Deal - Organisation,Deal - Methodology,Deal - Sample,Deal - Industry,Deal - Value,Deal - Research Type,level_7,Deal - Location
2,Sample for Postal Measurement Panel - Netherlands,Kantar,CAWI,318.0,B2C,0,Quantitative,0,Netherlands
7,Multi-country study in Middle East qual,IRB,IDI,0.0,B2C,0,Qualitative,0,Saudi Arabia
8,Multi-country study in Middle East qual,IRB,IDI,0.0,B2C,0,Qualitative,1,United Arab Emirates
9,Multi-country study in Middle East qual,IRB,IDI,0.0,B2C,0,Qualitative,2,Qatar
10,Multi-country study in Middle East qual,IRB,IDI,0.0,B2C,0,Qualitative,3,Kuwait
...,...,...,...,...,...,...,...,...,...
184,P22-65826,Market xcel,OBB,150.0,B2C,0,Qualitative,0,United States of America
185,Fwd: [Partner] Demande de Feas-devis/ Projet N...,Bilendi,CAWI,3300.0,B2C,0,Quantitative,0,Luxembourg
189,BA151: Pricing Request: Skin Care Request: Sma...,Borderless Access,FGD,6.0,B2C,0,Qualitative,0,Mexico
190,P22-66060,Market xcel,"Diary Study, CAWI",300.0,B2C,51000,Quantitative,0,United States of America


In [ ]:
# Agrupando 

grouped_df_b2c = df_b2c.groupby('Deal - Location')['Deal - Title'].count().reset_index()
grouped_df_b2c.rename(columns={"Deal - Title": "Industry - B2C"}, inplace=True)
grouped_df_b2c

,Deal - Location,Industry - B2C
0,Argentina,1
1,Australia,2
2,Azerbaijan,2
3,Bahamas,2
4,Belice,1
5,Brazil,3
6,Canada,2
7,Chile,1
8,China,3
9,Egypt,1


In [ ]:
# Fazendo o merge com o DF agrupado pelo número de projetos perdidos

b2c = pd.merge(grouped, grouped_df_b2c,on='Deal - Location',how='left')
b2c

,Deal - Location,Deal - Title,Industry - B2C
0,Argentina,1,1.0
1,Australia,7,2.0
2,Azerbaijan,2,2.0
3,Bahamas,2,2.0
4,Belgium,1,NaN
5,Belice,1,1.0
6,Brazil,5,3.0
7,Canada,6,2.0
8,Chile,2,1.0
9,China,6,3.0


## B2B

In [ ]:
# Filtrando por indústria B2B

df_b2b = new_df[new_df['Deal - Industry'].str.contains("B2B", na=False)]

In [ ]:
# Agrupando por localização

grouped_df_b2b = df_b2b.groupby('Deal - Location')['Deal - Title'].count().reset_index()
grouped_df_b2b.rename(columns={"Deal - Title": "Industry - B2B"}, inplace=True)
grouped_df_b2b

,Deal - Location,Industry - B2B
0,Australia,4
1,Belgium,1
2,Brazil,1
3,Canada,3
4,Chile,1
5,China,1
6,Costa Rica,1
7,Egypt,1
8,France,5
9,Germany,3


In [ ]:
all = pd.merge(grouped_df_b2c, grouped_df_b2b,on='Deal - Location',how='left')
all

,Deal - Location,Industry - B2C,Industry - B2B
0,Argentina,1,NaN
1,Australia,2,4.0
2,Azerbaijan,2,NaN
3,Bahamas,2,NaN
4,Belice,1,NaN
5,Brazil,3,1.0
6,Canada,2,3.0
7,Chile,1,1.0
8,China,3,1.0
9,Egypt,1,1.0


In [ ]:
all_2 = pd.merge(grouped, all,on='Deal - Location',how='left')
all_2

,Deal - Location,Deal - Title,Industry - B2C,Industry - B2B
0,Argentina,1,1.0,NaN
1,Australia,7,2.0,4.0
2,Azerbaijan,2,2.0,NaN
3,Bahamas,2,2.0,NaN
4,Belgium,1,NaN,NaN
5,Belice,1,1.0,NaN
6,Brazil,5,3.0,1.0
7,Canada,6,2.0,3.0
8,Chile,2,1.0,1.0
9,China,6,3.0,1.0


## Healthcare

In [ ]:
def filtro(termo):
  ds = new_df[new_df['Deal - Industry'].str.contains(termo, na=False)]
  ds = ds.groupby('Deal - Location')['Deal - Title'].count().reset_index()
  ds.rename(columns={"Deal - Title": termo}, inplace=True)

  
  return ds

In [ ]:
health = filtro('Healthcare') 
health

,Deal - Location,Healthcare
0,Australia,1
1,Canada,1
2,China,2
3,Colombia,1
4,Dominican Republic,1
5,England,1
6,France,3
7,Germany,4
8,India,1
9,Indonesia,1


In [ ]:
all_3 = pd.merge(all_2, health,on='Deal - Location',how='left')
all_3

,Deal - Location,Deal - Title,Industry - B2C,Industry - B2B,Healthcare
0,Argentina,1,1.0,NaN,NaN
1,Australia,7,2.0,4.0,1.0
2,Azerbaijan,2,2.0,NaN,NaN
3,Bahamas,2,2.0,NaN,NaN
4,Belgium,1,NaN,NaN,NaN
5,Belice,1,1.0,NaN,NaN
6,Brazil,5,3.0,1.0,NaN
7,Canada,6,2.0,3.0,1.0
8,Chile,2,1.0,1.0,NaN
9,China,6,3.0,1.0,2.0


## Artificial Intelligence	

In [ ]:
ai = filtro('Artificial Intelligence')
ai

,Deal - Location,Artificial Intelligence
0,Australia,1
1,China,1
2,France,1
3,Germany,1
4,Hong Kong,1
5,India,1
6,Japan,1
7,Spain,1
8,United States of America,1


In [ ]:
all_4 = pd.merge(all_3, ai,on='Deal - Location',how='left')
all_4

,Deal - Location,Deal - Title,Industry - B2C,Industry - B2B,Healthcare,Artificial Intelligence
0,Argentina,1,1.0,NaN,NaN,NaN
1,Australia,7,2.0,4.0,1.0,1.0
2,Azerbaijan,2,2.0,NaN,NaN,NaN
3,Bahamas,2,2.0,NaN,NaN,NaN
4,Belgium,1,NaN,NaN,NaN,NaN
5,Belice,1,1.0,NaN,NaN,NaN
6,Brazil,5,3.0,1.0,NaN,NaN
7,Canada,6,2.0,3.0,1.0,NaN
8,Chile,2,1.0,1.0,NaN,NaN
9,China,6,3.0,1.0,2.0,1.0


## B2B, B2C	

In [ ]:
bb = filtro('B2B, B2C')
bb

,Deal - Location,"B2B, B2C"
0,Australia,1
1,China,1
2,India,2
3,New Zealand,1
4,Thailand,1
5,United States of America,2


In [ ]:
all_5 = pd.merge(all_4, bb,on='Deal - Location',how='left')
all_5

,Deal - Location,Deal - Title,Industry - B2C,Industry - B2B,Healthcare,Artificial Intelligence,"B2B, B2C"
0,Argentina,1,1.0,NaN,NaN,NaN,NaN
1,Australia,7,2.0,4.0,1.0,1.0,1.0
2,Azerbaijan,2,2.0,NaN,NaN,NaN,NaN
3,Bahamas,2,2.0,NaN,NaN,NaN,NaN
4,Belgium,1,NaN,NaN,NaN,NaN,NaN
5,Belice,1,1.0,NaN,NaN,NaN,NaN
6,Brazil,5,3.0,1.0,NaN,NaN,NaN
7,Canada,6,2.0,3.0,1.0,NaN,NaN
8,Chile,2,1.0,1.0,NaN,NaN,NaN
9,China,6,3.0,1.0,2.0,1.0,1.0


## Testing/Package Goods

In [ ]:
testing = filtro('Testing/Package Goods')
testing

,Deal - Location,Testing/Package Goods
0,Indonesia,1


In [ ]:
all_6 = pd.merge(all_5, testing,on='Deal - Location',how='left')
all_6

,Deal - Location,Deal - Title,Industry - B2C,Industry - B2B,Healthcare,Artificial Intelligence,"B2B, B2C",Testing/Package Goods
0,Argentina,1,1.0,NaN,NaN,NaN,NaN,NaN
1,Australia,7,2.0,4.0,1.0,1.0,1.0,NaN
2,Azerbaijan,2,2.0,NaN,NaN,NaN,NaN,NaN
3,Bahamas,2,2.0,NaN,NaN,NaN,NaN,NaN
4,Belgium,1,NaN,NaN,NaN,NaN,NaN,NaN
5,Belice,1,1.0,NaN,NaN,NaN,NaN,NaN
6,Brazil,5,3.0,1.0,NaN,NaN,NaN,NaN
7,Canada,6,2.0,3.0,1.0,NaN,NaN,NaN
8,Chile,2,1.0,1.0,NaN,NaN,NaN,NaN
9,China,6,3.0,1.0,2.0,1.0,1.0,NaN


# Tipo de Pesquisa

## Quants

In [ ]:
def filtro_tipo(termo):
  ds = new_df[new_df['Deal - Research Type'].str.contains(termo, na=False)]
  ds = ds.groupby('Deal - Location')['Deal - Title'].count().reset_index()
  ds.rename(columns={"Deal - Title": termo}, inplace=True)

  
  return ds

In [ ]:
quant = filtro_tipo('Quantitative')
quant

,Deal - Location,Quantitative
0,Australia,2
1,Azerbaijan,2
2,Bahamas,2
3,Belice,1
4,Brazil,2
5,Canada,2
6,Chile,1
7,China,1
8,Colombia,1
9,Costa Rica,1


In [ ]:
all_7 = pd.merge(all_6, quant,on='Deal - Location',how='left')
all_7

,Deal - Location,Deal - Title,Industry - B2C,Industry - B2B,Healthcare,Artificial Intelligence,"B2B, B2C",Testing/Package Goods,Quantitative
0,Argentina,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Australia,7,2.0,4.0,1.0,1.0,1.0,NaN,2.0
2,Azerbaijan,2,2.0,NaN,NaN,NaN,NaN,NaN,2.0
3,Bahamas,2,2.0,NaN,NaN,NaN,NaN,NaN,2.0
4,Belgium,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Belice,1,1.0,NaN,NaN,NaN,NaN,NaN,1.0
6,Brazil,5,3.0,1.0,NaN,NaN,NaN,NaN,2.0
7,Canada,6,2.0,3.0,1.0,NaN,NaN,NaN,2.0
8,Chile,2,1.0,1.0,NaN,NaN,NaN,NaN,1.0
9,China,6,3.0,1.0,2.0,1.0,1.0,NaN,1.0


## Quals

In [ ]:
qual = filtro_tipo('Qualitative')
qual

,Deal - Location,Qualitative
0,Argentina,1
1,Australia,5
2,Belgium,1
3,Brazil,3
4,Canada,3
5,Chile,1
6,China,5
7,Dominican Republic,1
8,Egypt,2
9,France,8


In [ ]:
all_8 = pd.merge(all_7, qual,on='Deal - Location',how='left')
all_8

,Deal - Location,Deal - Title,Industry - B2C,Industry - B2B,Healthcare,Artificial Intelligence,"B2B, B2C",Testing/Package Goods,Quantitative,Qualitative
0,Argentina,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,Australia,7,2.0,4.0,1.0,1.0,1.0,NaN,2.0,5.0
2,Azerbaijan,2,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN
3,Bahamas,2,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN
4,Belgium,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,Belice,1,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
6,Brazil,5,3.0,1.0,NaN,NaN,NaN,NaN,2.0,3.0
7,Canada,6,2.0,3.0,1.0,NaN,NaN,NaN,2.0,3.0
8,Chile,2,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0
9,China,6,3.0,1.0,2.0,1.0,1.0,NaN,1.0,5.0


# Gráficos

In [ ]:
all_8.rename(columns={"Deal - Title": "Projetos_Perdidos"},inplace=True)
all_8.rename(columns={"Industry - B2C": "B2C"},inplace=True)
all_8.rename(columns={"Industry - B2B": "B2B"},inplace=True)

In [ ]:
all_8.fillna(0, inplace = True)
all_8

,Deal - Location,Projetos_Perdidos,B2C,B2B,Healthcare,Artificial Intelligence,"B2B, B2C",Testing/Package Goods,Quantitative,Qualitative
0,Argentina,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Australia,7,2.0,4.0,1.0,1.0,1.0,0.0,2.0,5.0
2,Azerbaijan,2,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,Bahamas,2,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
4,Belgium,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,Belice,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,Brazil,5,3.0,1.0,0.0,0.0,0.0,0.0,2.0,3.0
7,Canada,6,2.0,3.0,1.0,0.0,0.0,0.0,2.0,3.0
8,Chile,2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
9,China,6,3.0,1.0,2.0,1.0,1.0,0.0,1.0,5.0


In [ ]:
def inteirar(termo):
  all_8[termo] = all_8[termo].astype('int')

  return all_8 


In [ ]:
inteirar('B2C')
inteirar('B2B')
inteirar('Healthcare')
inteirar('Artificial Intelligence')
inteirar('B2B, B2C')
inteirar('Testing/Package Goods')
inteirar('Quantitative')
inteirar('Qualitative')

,Deal - Location,Projetos_Perdidos,B2C,B2B,Healthcare,Artificial Intelligence,"B2B, B2C",Testing/Package Goods,Quantitative,Qualitative
0,Argentina,1,1,0,0,0,0,0,0,1
1,Australia,7,2,4,1,1,1,0,2,5
2,Azerbaijan,2,2,0,0,0,0,0,2,0
3,Bahamas,2,2,0,0,0,0,0,2,0
4,Belgium,1,0,0,0,0,0,0,0,1
5,Belice,1,1,0,0,0,0,0,1,0
6,Brazil,5,3,1,0,0,0,0,2,3
7,Canada,6,2,3,1,0,0,0,2,3
8,Chile,2,1,1,0,0,0,0,1,1
9,China,6,3,1,2,1,1,0,1,5


In [ ]:
# Identificando latitude e longitude de cada localização para plotar no gráfico do mapa.

geolocator = Nominatim(user_agent="geoapiExercises")

def get_lat_long(location):
  location = geolocator.geocode(location)
  return location.latitude, location.longitude

all_8['latitude'], all_8['longitude'] = zip(*all_8['Deal - Location'].apply(get_lat_long))
all_8

,Deal - Location,Projetos_Perdidos,B2C,B2B,Healthcare,Artificial Intelligence,"B2B, B2C",Testing/Package Goods,Quantitative,Qualitative,latitude,longitude
0,Argentina,1,1,0,0,0,0,0,0,1,-34.996496,-64.967282
1,Australia,7,2,4,1,1,1,0,2,5,-24.776109,134.755000
2,Azerbaijan,2,2,0,0,0,0,0,2,0,40.393629,47.787251
3,Bahamas,2,2,0,0,0,0,0,2,0,24.773655,-78.000055
4,Belgium,1,0,0,0,0,0,0,0,1,50.640281,4.666715
5,Belice,1,1,0,0,0,0,0,1,0,16.825979,-88.760093
6,Brazil,5,3,1,0,0,0,0,2,3,-10.333333,-53.200000
7,Canada,6,2,3,1,0,0,0,2,3,61.066692,-107.991707
8,Chile,2,1,1,0,0,0,0,1,1,-31.761336,-71.318770
9,China,6,3,1,2,1,1,0,1,5,35.000074,104.999927


In [ ]:
import plotly.graph_objects as go

fig = px.scatter_mapbox(all_8, lat='latitude', lon='longitude', size='Projetos_Perdidos', hover_name = 'Deal - Location',
                        hover_data = ['B2C','B2B','Healthcare','Artificial Intelligence','B2B, B2C','Testing/Package Goods','Quantitative','Qualitative'],
                        color_continuous_scale='inferno', size_max=40,
                        mapbox_style='open-street-map', zoom=1)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
@interact(country=gro_sor['Deal - Location'])
def country(country):
  filtered = all_8[all_8['Deal - Location'].str.contains(country)]
  filtered = filtered.melt(id_vars='Deal - Location', value_vars=['B2C','B2B','Healthcare','Artificial Intelligence','B2B','B2C','Testing/Package Goods','Quantitative','Qualitative'], var_name='Project_Type')
  bar = px.bar(filtered, x='Project_Type', y='value',title=country)

  return bar

interactive(children=(Dropdown(description='country', options=('United States of America', 'France', 'Germany'…

In [ ]:
@interact(country=gro_sor['Deal - Location'])
def country(country):
  filtered = all_8[all_8['Deal - Location'].str.contains(country)]
  filtered = filtered.melt(id_vars='Deal - Location', value_vars=['B2C','B2B','Healthcare','Artificial Intelligence','B2B','B2C','Testing/Package Goods','Quantitative','Qualitative'], var_name='Project_Type')
  bar2 = px.bar(filtered, x='Project_Type', y='value',title=country)

  return bar2

# Metodologia

In [ ]:
# Agrupando e contando o número de projetos perdidos por metodologia.

grouped_2 = new_df.groupby("Deal - Methodology")["Deal - Title"].count().reset_index()

In [ ]:
grouped_2.sort_values('Deal - Title', ascending=False)

In [ ]:
# Agrupando e contando o número de projetos perdidos por tipo de pesquisa (quant ou qual).

grouped_3 = new_df.groupby("Deal - Research Type")["Deal - Title"].count().reset_index()
grouped_3.sort_values('Deal - Title', ascending=False)

,Deal - Research Type,Deal - Title
0,Qualitative,113
2,Quantitative,66
1,"Qualitative, Quantitative",9


In [ ]:
# Agrupando e contando o número de projetos perdidos por Indústria.

grouped_4 = new_df.groupby("Deal - Industry")["Deal - Title"].count().reset_index()
grouped_4.sort_values('Deal - Title', ascending=False)

,Deal - Industry,Deal - Title
3,B2C,80
4,Healthcare,47
1,B2B,46
0,Artificial Intelligence,9
2,"B2B, B2C",8
5,Testing/Package Goods,1
